# SWB analysis pipeline part 2: 

- aggregate CPE epoch power data + metadata + roi locs 
- make dataframes for trial averaged regressions for hfa + beta 
- format time-resolved tfr data for TFR x condition plots 
- concat tfr ROI data across subjects 


*Created: 04/08/24* \
*Updated: 06/24/2024*


In [1]:
import numpy as np
import mne
from glob import glob
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import seaborn as sns
from scipy.stats import zscore, linregress, ttest_ind, ttest_rel, ttest_1samp, pearsonr, spearmanr
import pandas as pd
from mne.preprocessing.bads import _find_outliers
import os 
import joblib
import re
import datetime
import scipy
import random
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.mixed_linear_model import MixedLM 
from joblib import Parallel, delayed
import pickle
import itertools
import time 
from matplotlib.ticker import StrMethodFormatter



import warnings
warnings.filterwarnings('ignore')




In [4]:
# Specify root directory for un-archived data and results 
base_dir   = '/sc/arion/projects/guLab/Alie/SWB/'
anat_dir   = f'{base_dir}ephys_analysis/recon_labels/'
neural_dir = f'{base_dir}ephys_analysis/data/'
behav_dir  = f'{base_dir}swb_behav_models/data/behavior_preprocessed/'
save_dir   = f'{base_dir}ephys_analysis/results/regression_data/'
script_dir = '/hpc/users/finka03/swb_ephys_analysis/scripts/'

date = datetime.date.today().strftime('%m%d%Y')
print(date)

06242024


In [5]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'{base_dir}ephys_analysis/LFPAnalysis/')

from LFPAnalysis import analysis_utils

sys.path.append(f'{script_dir}analysis_notebooks/')

from ieeg_tools import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
subj_ids = list(pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[0]).PatientID)
n_subj = len(subj_ids)
# subj_ids


In [7]:
bdi_list = pd.read_excel(f'{base_dir}ephys_analysis/subj_info/SWB_subjects.xlsx', sheet_name='Usable_Subjects', usecols=[3])
bdi_list = list(bdi_list.SWB_BDI)
subj_info_df = pd.DataFrame({'subj_id':subj_ids,'bdi':bdi_list})
subj_info_df



,subj_id,bdi
0,MS002,14
1,MS003,8
2,MS009,16
3,MS011,13
4,MS015,26
5,MS016,10
6,MS017,26
7,MS019,12
8,MS022,10
9,MS024,16


In [13]:
analysis_freqs = {'delta':[2,4],'theta':[4,9],'alpha':[9,13],'beta':[13,30],'gamma':[30,70],'hfa':[70,200]}
analysis_freqs


{'delta': [2, 4],
 'theta': [4, 9],
 'alpha': [9, 13],
 'beta': [13, 30],
 'gamma': [30, 70],
 'hfa': [70, 200]}

In [15]:
all_subj_power     = {}
all_subj_metadata  = {}


# load pow epochs data 
for subj_id in subj_ids: 
    
    # load cpe power epochs for single subject 
    power_epochs = mne.time_frequency.read_tfrs(f'{neural_dir}{subj_id}/{subj_id}_CpeOnset-tfr.h5')[0]
    all_subj_metadata[subj_id] = power_epochs.metadata.copy()

    # convert mne power data to data frame
    power_df = power_epochs.to_data_frame()
    del power_epochs # remove power epochs from memory to speed up loop
    
    subj_band_dfs = {}
    # extract power df with mean freq-specific power for each epoch for each elec 
    for band,freq_range in analysis_freqs.items():
        band_df = power_df[(power_df.freq >= freq_range[0])&(power_df.freq <= freq_range[1])].groupby(['epoch']).agg('mean').reset_index()
        band_df['band'] = band
        subj_band_dfs[band] = band_df 
    del power_df # delete full power df to save memory
    all_subj_power[subj_id] = subj_band_dfs
    
# create a binary pickle file 
pickle.dump(all_subj_power,
            open(f'{save_dir}all_subj_power_{("_").join(list(analysis_freqs.keys()))}_{date}.pkl',"wb"))

# # # create a binary pickle file 
pickle.dump(all_subj_metadata,
            open(f'{save_dir}all_subj_metadata.pkl',"wb")) 

Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS002/MS002_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS003/MS003_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS009/MS009_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS011/MS011_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS015/MS015_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS016/MS016_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS017/MS017_CpeOnset-tfr.h5 ...
Adding metadata with 19 columns
Reading /sc/arion/projects/guLab/Alie/SWB/ephys_analysis/data/MS019/MS019_CpeOnset-tfr.h5 ...
Adding metadata with 19 

# Make Regression DFs

1. Update metadata vars - add t+1 vars, normalize vecs by 2std
2. Aggregate metadata, power data, ROI info, bdi info into one master df 

In [16]:
### update metadata - zscore continuous regressors + add RT info
for subj_id in subj_ids:
    subj_df         = all_subj_metadata[subj_id]
    rounds_t1       = subj_df.Round.tolist()[1:]
    logRT_list      = np.log(subj_df['RT']).tolist() # take log of RT
    logRT_t1_list   = logRT_list[1:] # get logRT_t1
    GambleChoice_t1 = subj_df.GambleChoice.tolist()[1:]
    GambleEV_t1     = subj_df.GambleEV.tolist()[1:]
    TrialType_t1    = subj_df.TrialType.tolist()[1:]
    TrialEV         = subj_df.GambleEV - subj_df.SafeBet
    TrialEV_t1      = TrialEV[1:].tolist()
    rounds_t1.append(np.nan)
    logRT_t1_list.append(np.nan) # add nan to logRT_t1 (no t1 for last round)
    GambleChoice_t1.append(np.nan)
    GambleEV_t1.append(np.nan)
    TrialType_t1.append(np.nan) 
    TrialEV_t1.append(np.nan)
    subj_df['logRT']           = logRT_list
    subj_df['logRT_t1']        = logRT_t1_list
    subj_df['Round_t1']        = rounds_t1
    subj_df['GambleChoice_t1'] = GambleChoice_t1
    subj_df['GambleEV_t1']     = GambleEV_t1
    subj_df['TrialType_t1']    = TrialType_t1
    subj_df['TrialEV']         = TrialEV
    subj_df['TrialEV_t1']      = TrialEV_t1
    subj_df['bdi']             = subj_info_df.bdi[subj_info_df.subj_id==subj_id].values[0]
    subj_df['subj_id']         = subj_id
    
    # update metadata
    all_subj_metadata[subj_id] = subj_df
    

In [44]:
# # # create a binary pickle file 
pickle.dump(all_subj_metadata,
            open(f'{save_dir}all_subj_metadata_clean.pkl',"wb")) 

In [20]:
# round_test = [list(zip(all_subj_metadata[key].Round.tolist(),all_subj_metadata[key].RT.tolist(),all_subj_metadata[key].logRT.tolist(),all_subj_metadata[key].logRT_t1.tolist())) for key in all_subj_metadata.keys()]
# is_consecutive = [all([all_subj_metadata[subj_id].Round[i] == all_subj_metadata[subj_id].Round_t1[i-1] for i in range(1, len(all_subj_metadata[subj_id].Round))]) for subj_id in all_subj_metadata.keys()]
# is_consecutive
# round_test


In [22]:
### load ROI reref master 
roi_reref_labels_master_df = pd.read_csv(glob(f'{base_dir}ephys_analysis/results/roi_info/roi_reref_labels_master.csv')[0])
roi_reref_labels_master_df = roi_reref_labels_master_df.drop(columns=['Unnamed: 0'])
roi_reref_labels_master_df



,subj_id,reref_ch_names,ch_label4roi,ch_type4roi,loc4roi,mni_x,mni_y,mni_z,hemi,roi
0,MS002,lacas1-lacas2,lacas1,anode,left cingulate gyrus d,-6.382462,37.158688,-3.130044,l,acc
1,MS002,lacas2-lacas3,lacas2,anode,left cingulate gyrus e,-6.368174,38.606223,2.270621,l,acc
2,MS002,lacas3-lacas4,lacas3,anode,left cingulate gyrus f,-6.390079,39.941566,7.640265,l,acc
3,MS002,lacas4-lacas5,lacas4,anode,left cingulate gyrus f,-6.914519,41.546899,12.993427,l,acc
4,MS002,lacas5-lacas6,lacas5,anode,left cingulate gyrus g,-6.933604,42.745784,18.267675,l,acc
...,...,...,...,...,...,...,...,...,...,...
1875,DA039,rtp6-rtp7,rtp6,anode,right superior middle temporal pole d,40.532268,7.354525,-33.253436,r,temporal pole
1876,DA039,rtp7-rtp8,rtp7,anode,right superior middle temporal pole d,43.911226,7.341369,-31.915085,r,temporal pole
1877,DA039,rsgcc6-rsgcc7,rsgcc7,cathode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc
1878,DA039,rsgcc7-rsgcc8,rsgcc7,anode,right anterior pars triangularis b,42.378987,31.277597,2.471854,r,vlpfc


In [39]:
regression_band_data = {f'{freq}':[] for freq in list(analysis_freqs.keys())}


In [40]:
continuous_regressors = ['TrialEV','TrialEV_t1','GambleEV','GambleEV_t1','SafeBet',
                         'TotalProfit','CR', 'choiceEV', 'RPE','Profit',
                         'decisionCPE', 'decisionRegret','decisionRelief']

In [42]:
for band in list(analysis_freqs.keys()):
    
    
    band_master_df = []
    
    for subj_id in subj_ids: 
        pow_df = all_subj_power[subj_id][band].copy()

        # reshape pow_df 
        pow_reg_df = pow_df.melt(id_vars=['epoch','freq','band'],
                                    value_vars=pow_df.columns.drop(['epoch','time','freq']).tolist().copy(),
                                    var_name='reref_ch_names', value_name='band_pow',
                                    ignore_index = False)

        pow_reg_df['subj_id'] = subj_id # add subj_id to df values
        # reshape metadata df 
        metadata_df = all_subj_metadata[subj_id].reset_index()
        metadata_df = metadata_df.rename(columns={'index':'epoch'})

        # merge pow and metadata dfs keeping epoch as index 
        subj_regression_df = pd.merge(pow_reg_df, metadata_df,left_on=['epoch','subj_id'],
                                    right_on=['epoch','subj_id']).reset_index(drop=True).set_index('epoch')
        subj_regression_df['epoch_num'] = subj_regression_df.index # keep epoch as a column

        # add roi info 
        subj_epochs = pow_reg_df.epoch.unique()
        # extract roi_reref_labels_df for each subj 
        subj_roi_reref_labels = roi_reref_labels_master_df[roi_reref_labels_master_df['subj_id']==subj_id]
        # add epochs to df - repeat each elec id for every epoch to join with master df 
        subj_roi_reref_labels = (pd.concat([subj_roi_reref_labels]*len(subj_epochs),
                   keys = subj_epochs,
                   names = ['epoch',None])).reset_index(level=1,drop=True) #make sure epochs are the index
        subj_roi_reref_labels['epoch_num'] = subj_roi_reref_labels.index

        # create unique reref ch id for regression
        subj_roi_reref_labels['unique_reref_ch'] = subj_roi_reref_labels[['subj_id', 'reref_ch_names']].agg('_'.join, axis=1)

        # merge regression df with roi reref info df  
        subj_regression_df = pd.merge(subj_regression_df, subj_roi_reref_labels,left_on=
                                      ['subj_id','reref_ch_names','epoch_num'],right_on=
                                      ['subj_id','reref_ch_names','epoch_num']).reset_index(
                                        drop=True).set_index('epoch_num')
        
        subj_regression_df['epoch'] = subj_regression_df.index
        

        ### cleaning within subject BEFORE zscoring regressors! 
        # remove trials where RT is too fast (random choice)
        subj_regression_df = subj_regression_df[subj_regression_df.RT>0.3].reset_index(drop=True)
        # remove trials where there is no choice at trial t
        subj_regression_df = subj_regression_df[(subj_regression_df.GambleChoice=='gamble') | 
                                                (subj_regression_df.GambleChoice == 'safe')].reset_index(drop=True)
        # remove trials where there is no choice for trial t1 (RT_t1 is nonsense)
        subj_regression_df = subj_regression_df[(subj_regression_df.GambleChoice_t1=='gamble') | 
                                                (subj_regression_df.GambleChoice_t1 == 'safe')].reset_index(drop=True)
        # remove trials where there is no outcome
        subj_regression_df = subj_regression_df[(subj_regression_df.Outcome=='good') | 
                                                (subj_regression_df.Outcome == 'bad')].reset_index(drop=True)
        
        # remove epoch 76 bc photodiode times are not correct at breakpoint 
        subj_regression_df = subj_regression_df[subj_regression_df.Round != 76].reset_index(drop=True)
        # remove trials where logRT_t1 is inf or -inf (means RT = 0)
        subj_regression_df = subj_regression_df[~np.isinf(subj_regression_df.logRT_t1)].reset_index(drop=True)
        # format GambleChoice + TrialType vars as categories 
        subj_regression_df['GambleChoice']    = subj_regression_df['GambleChoice'].astype('category')
        subj_regression_df['GambleChoice_t1'] = subj_regression_df['GambleChoice_t1'].astype('category')
        subj_regression_df['TrialType']       = subj_regression_df['TrialType'].astype('category')
        subj_regression_df['TrialType_t1']    = subj_regression_df['TrialType_t1'].astype('category')
        # make sure Round_t1 is an integer
        subj_regression_df['Round_t1']        = subj_regression_df['Round_t1'].astype('Int64')
        
        # zscore continuous regressors AFTER removing bad trials!! 
        for reg in continuous_regressors:
            subj_regression_df[reg] = norm_zscore(subj_regression_df[reg].values) # zscore is /2std


        band_master_df.append(subj_regression_df)
        
        del subj_regression_df,pow_df, pow_reg_df,metadata_df,subj_roi_reref_labels

    
    
    band_master_df = pd.concat(band_master_df).reset_index(drop=True)
    # reorder columns 
    band_master_df = band_master_df[['subj_id','bdi','reref_ch_names','unique_reref_ch','epoch','freq','band',
                                     'band_pow','roi','hemi','ch_label4roi','ch_type4roi','loc4roi',
                                     'logRT_t1','logRT','RT','Round', 'Round_t1','TrialType','TrialType_t1',
                                     'GambleEV', 'GambleEV_t1','SafeBet','GambleChoice','GambleChoice_t1','TrialEV',                                 
                                     'Profit','TotalProfit','RPE','decisionCPE', 'decisionRegret','decisionRelief']]

    band_master_df.to_csv(f'{save_dir}{band}_master_df.csv')
    regression_band_data[band] = band_master_df
    del band_master_df




In [43]:
regression_band_data['theta']

,subj_id,bdi,reref_ch_names,unique_reref_ch,epoch,freq,band,band_pow,roi,hemi,...,SafeBet,GambleChoice,GambleChoice_t1,TrialEV,Profit,TotalProfit,RPE,decisionCPE,decisionRegret,decisionRelief
0,MS002,14,lacas1-lacas2,MS002_lacas1-lacas2,0,6.232769,theta,-0.243781,acc,l,...,-0.007878,safe,gamble,-0.437011,-0.032900,-1.028523,-0.008746,-0.395746,-0.287638,-0.392122
1,MS002,14,lacas2-lacas3,MS002_lacas2-lacas3,0,6.232769,theta,-0.178626,acc,l,...,-0.007878,safe,gamble,-0.437011,-0.032900,-1.028523,-0.008746,-0.395746,-0.287638,-0.392122
2,MS002,14,lacas3-lacas4,MS002_lacas3-lacas4,0,6.232769,theta,-0.249534,acc,l,...,-0.007878,safe,gamble,-0.437011,-0.032900,-1.028523,-0.008746,-0.395746,-0.287638,-0.392122
3,MS002,14,lacas4-lacas5,MS002_lacas4-lacas5,0,6.232769,theta,-0.000454,acc,l,...,-0.007878,safe,gamble,-0.437011,-0.032900,-1.028523,-0.008746,-0.395746,-0.287638,-0.392122
4,MS002,14,lacas5-lacas6,MS002_lacas5-lacas6,0,6.232769,theta,-0.265047,acc,l,...,-0.007878,safe,gamble,-0.437011,-0.032900,-1.028523,-0.008746,-0.395746,-0.287638,-0.392122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
258468,DA039,22,rtp6-rtp7,DA039_rtp6-rtp7,148,6.232769,theta,0.031661,temporal pole,r,...,-0.047267,safe,gamble,-0.359505,-0.042732,0.993053,0.022985,-0.272392,-0.115665,-0.334244
258469,DA039,22,rtp7-rtp8,DA039_rtp7-rtp8,148,6.232769,theta,-0.187044,temporal pole,r,...,-0.047267,safe,gamble,-0.359505,-0.042732,0.993053,0.022985,-0.272392,-0.115665,-0.334244
258470,DA039,22,rsgcc6-rsgcc7,DA039_rsgcc6-rsgcc7,148,6.232769,theta,-0.059029,vlpfc,r,...,-0.047267,safe,gamble,-0.359505,-0.042732,0.993053,0.022985,-0.272392,-0.115665,-0.334244
258471,DA039,22,rsgcc7-rsgcc8,DA039_rsgcc7-rsgcc8,148,6.232769,theta,0.055702,vlpfc,r,...,-0.047267,safe,gamble,-0.359505,-0.042732,0.993053,0.022985,-0.272392,-0.115665,-0.334244
